In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
sat_speech_set_1000_prepared = dataiku.Dataset("sat_speech_set_1000_prepared")
sat_speech_set_1000_prepared_df = sat_speech_set_1000_prepared.get_dataframe()
negation_Lexique = dataiku.Dataset("negation_Lexique")
negation_Lexique_df = negation_Lexique.get_dataframe()


liste_de_negation = list(negation_Lexique_df['Negation'])

liste_convers = list(sat_speech_set_1000_prepared_df['text_diarise'])
liste_convers = liste_convers

# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

#sat_nagation_all_df = ... # Compute a Pandas dataframe to write into sat_nagation_all


# Write recipe outputs
#sat_nagation_all = dataiku.Dataset("sat_nagation_all")
#sat_nagation_all.write_with_schema(sat_nagation_all_df)

In [ ]:
liste_negation = []
for i in liste_de_negation:
    negation = str(i)
    negation = ' '+negation+' '
    liste_negation.append(negation)
    
liste_negation.remove(" n' ")  
liste_negation.append(" n'")
#print(liste_negation)

In [ ]:
liste_des_tours_de_paroles_par_conversation = []
for conversation in liste_convers:
    conversation=str(conversation)
    conversation_intermediaire = conversation.split(".")
    liste_des_tours_de_paroles_par_conversation.append(conversation_intermediaire)
print(len(liste_des_tours_de_paroles_par_conversation), 'conservations')#récupère la liste des conversations entières
#print(len(liste_des_tours_de_paroles_par_conversation[2]))

In [ ]:
import re
liste_oper_all = []
for i in range(len(liste_des_tours_de_paroles_par_conversation)):
    liste_operateur = []

    for tdp_conv in liste_des_tours_de_paroles_par_conversation[i]:
        conv_1 = str(tdp_conv)

        matchObj = re.search('Opérateur :.+', conv_1)
        if matchObj:
            liste_operateur.append(matchObj.group())
    liste_oper_all.append(liste_operateur)

In [ ]:
liste_client_all = []
for i in range(len(liste_des_tours_de_paroles_par_conversation)):
    liste_client = []

    for tdp_conv in liste_des_tours_de_paroles_par_conversation[i]:
        conv_1 = str(tdp_conv)

        matchObj = re.search('Client :.+', conv_1)
        if matchObj:
            liste_client.append(matchObj.group())
    liste_client_all.append(liste_client)
    


In [ ]:
def liste_negation(liste):
    
    liste_nega_all = []
    for i in range(len(liste)):
        #liste_nega = []
        convers = liste[i]

        liste_nega = []
        for tdp in convers:

            for ne in liste_de_negation:
                matchObj = re.search(ne, tdp)
                if matchObj:

                    #if tdp not in liste_nega:
                        liste_nega.append(tdp)

        liste_nega = list(set(liste_nega))
        #print(liste_nega)

        #if liste_nega not in liste_nega_all:

        liste_nega_all.append(liste_nega)
    
    return liste_nega_all

In [ ]:
liste_nega_all_client =  liste_negation(liste_client_all)
liste_nega_all_operateur =  liste_negation(liste_oper_all)

In [ ]:
def pourcentage(liste):
    liste_pourcentage = []
    for i in range(len(liste_des_tours_de_paroles_par_conversation)):
        len_nega = len(liste[i])
        len_all = len(liste_des_tours_de_paroles_par_conversation[i])
        if len_all == 0:
            pourcentage == 0
            liste_pourcentage.append(pourcentage)
        else:
            pourcentage = (len_nega/len_all)*100
            liste_pourcentage.append(pourcentage)
    return liste_pourcentage

In [ ]:
liste_pourcentage_client = pourcentage(liste_nega_all_client)
liste_pourcentage_operateur = pourcentage(liste_nega_all_operateur)

In [ ]:
rfap = sat_speech_set_1000_prepared_df['rfap']
SATISFACTION_GLOBALE = sat_speech_set_1000_prepared_df['SATISFACTION_GLOBALE']
#text_negation = sat_speech_1_df['text_negation']
domaine_acte = sat_speech_set_1000_prepared_df['domaine_acte']
famille_acte = sat_speech_set_1000_prepared_df['famille_acte']
objet_acte = sat_speech_set_1000_prepared_df['objet_acte']

In [ ]:
new_df = pd.DataFrame(columns = ['rfap','SATISFACTION_GLOBALE','text_negation_client','text_negation_opérateur','taux_negation_client','taux_negation_opérateur','domaine_acte','famille_acte','objet_acte'])
new_df['rfap'] = rfap
new_df['SATISFACTION_GLOBALE'] = SATISFACTION_GLOBALE
new_df['text_negation_client'] = liste_nega_all_client
new_df['text_negation_opérateur'] = liste_nega_all_operateur
new_df['taux_negation_client'] = liste_pourcentage_client
new_df['taux_negation_opérateur'] = liste_pourcentage_operateur
new_df['domaine_acte'] = domaine_acte
new_df['objet_acte'] = objet_acte
new_df['famille_acte'] = famille_acte

In [ ]:
sat_nagation_all = dataiku.Dataset("sat_nagation_all")
sat_nagation_all.write_with_schema(new_df)